In [1]:
# Get L2 Revenue and post it to a database (csv in github for now)
import pandas as pd
import sys
import time
import json
sys.path.append("../helper_functions")
import web3py_utils as w3py
from web3 import Web3
from datetime import datetime
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()
op_node = os.environ["OP_NODE"]

In [2]:
# https://github.com/ethereum-optimism/optimism/blob/b86522036ad11a91de1d1dadb6805167add83326/specs/predeploys.md?plain=1#L50

# Name, contract, Proxy ABI, Implementation ABI
fee_vaults = [
    ['SequencerFeeVault','0x4200000000000000000000000000000000000011'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"l1FeeWallet","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
    ['BaseFeeVault','0x4200000000000000000000000000000000000019'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
    ['L1FeeVault','0x420000000000000000000000000000000000001A'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
]

chains_rpcs = [
        ['op mainnet', op_node],
        ['base', 'https://mainnet.base.org/'],
        ['zora network', 'https://rpc.zora.energy/'],
        ['public goods network', 'https://rpc.publicgoods.network/'],
]

In [3]:
# Calculate the method signature hash (this step is optional if you already know the hash)
method_signature = "totalProcessed()"
method_id = Web3.keccak(text=method_signature)[:4].hex()
# Verify the method ID (this step is also optional, but useful for debugging)
print(f"Method ID: {method_id}")

Method ID: 0x84411d65


In [4]:
data_arr = []

for chain in chains_rpcs:
    chain_name = chain[0]
    rpc = chain[1]
    w3_conn = Web3(Web3.HTTPProvider(rpc))
    # Get the timestamp of the latest block
    block_timestamp = w3_conn.eth.get_block('latest').timestamp
    block_number = w3_conn.eth.get_block('latest').number
    # Convert the UNIX timestamp to a human-readable format
    block_time = datetime.utcfromtimestamp(block_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    for vault in fee_vaults:
        vault_name = vault[0]
        vault_address = vault[1]
        # vault_abi = json.loads(vault[2])
        # implementation_abi = json.loads(vault[3])

        # Call the function directly using eth_call
        response = w3_conn.eth.call({
            'to': vault_address,
            'data': method_id
        })
        wei_balance = w3_conn.eth.get_balance(vault_address)
        # Decode the result (assuming the function returns a uint256)
        proxy_processed_wei = Web3.to_int(hexstr=response.hex())
        
        alltime_revenue_wei = proxy_processed_wei+wei_balance
        alltime_revenue_eth = alltime_revenue_wei/1e18

        print(chain_name + ' | ' + vault_name + ': ' \
              + str(proxy_processed_wei) + ' | bal: ' + str(wei_balance)\
              + ' | total eth: ' + str( (alltime_revenue_eth) )
              )
        
        tmp = pd.DataFrame(
                [[block_time, block_number, chain_name, vault_name, vault_address, alltime_revenue_eth]]
                ,columns =['block_time','block_number','chain_name','valut_name','vault_address','alltime_revenue_eth']
                )
        data_arr.append(tmp)
        time.sleep(1)

data_df = pd.concat(data_arr)

op mainnet | SequencerFeeVault: 1063857200132073207047 | bal: 82376100952118272230 | total eth: 1146.2333010841915
op mainnet | BaseFeeVault: 222912724643847907088 | bal: 20392456810467963761 | total eth: 243.30518145431589
op mainnet | L1FeeVault: 3399092561901994272487 | bal: 255457858211431307642 | total eth: 3654.550420113426
base | SequencerFeeVault: 0 | bal: 1592226812674042681250 | total eth: 1592.2268126740428
base | BaseFeeVault: 0 | bal: 313546600296213013621 | total eth: 313.546600296213
base | L1FeeVault: 0 | bal: 1024889444834256072415 | total eth: 1024.889444834256
zora network | SequencerFeeVault: 136818362420691232282 | bal: 8764192591641859593 | total eth: 145.5825550123331
zora network | BaseFeeVault: 0 | bal: 17556323329982 | total eth: 1.7556323329982e-05
zora network | L1FeeVault: 108743413917044646187 | bal: 8962951415768408784 | total eth: 117.70636533281305
public goods network | SequencerFeeVault: 0 | bal: 4924113878063664749 | total eth: 4.9241138780636655
pub

In [5]:
display(data_df)

,block_time,chain_name,valut_name,vault_address,alltime_revenue_eth
0,2023-09-01 21:33:33,op mainnet,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.146233e+03
0,2023-09-01 21:33:33,op mainnet,BaseFeeVault,0x4200000000000000000000000000000000000019,2.433052e+02
0,2023-09-01 21:33:33,op mainnet,L1FeeVault,0x420000000000000000000000000000000000001A,3.654550e+03
0,2023-09-01 21:33:39,base,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.592227e+03
0,2023-09-01 21:33:39,base,BaseFeeVault,0x4200000000000000000000000000000000000019,3.135466e+02
0,2023-09-01 21:33:39,base,L1FeeVault,0x420000000000000000000000000000000000001A,1.024889e+03
0,2023-09-01 21:33:43,zora network,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.455826e+02
0,2023-09-01 21:33:43,zora network,BaseFeeVault,0x4200000000000000000000000000000000000019,1.755632e-05
0,2023-09-01 21:33:43,zora network,L1FeeVault,0x420000000000000000000000000000000000001A,1.177064e+02
0,2023-09-01 21:33:47,public goods network,SequencerFeeVault,0x4200000000000000000000000000000000000011,4.924114e+00
